# MED: Morphological Encoder-Decoder

In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
!pip install six
!pip install tqdm
!pip install https://github.com/pytorch/text/archive/v0.2.3.zip
!pip install future

In [5]:
!git clone https://github.com/OpenNMT/OpenNMT-py.git
%cd OpenNMT-py
!ls

Cloning into 'OpenNMT-py'...
remote: Counting objects: 8434, done.
remote: Total 8434 (delta 0), reused 0 (delta 0), pack-reused 8434
Receiving objects: 100% (8434/8434), 80.94 MiB | 22.84 MiB/s, done.
Resolving deltas: 100% (5771/5771), done.
/content/OpenNMT-py
available_models			  LICENSE.md		server.py
CONTRIBUTING.md				  onmt			setup.py
data					  preprocess.py		test
Dockerfile				  README.md		tools
docs					  requirements.opt.txt	train.py
github_deploy_key_opennmt_opennmt_py.enc  requirements.txt	translate.py


In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls drive

In [0]:
%cd OpenNMT-py
!cp drive/MED/data/med_data_v2* data

In [0]:
!mkdir models

## Train German

Number of enc/dec layers: 2

In [31]:
!python train.py -data data/med_data_v2 \
                 -save_model models/med-model-bs64 \
                 -src_word_vec_size 300 \
                 -tgt_word_vec_size 300 \
                 -encoder_type brnn \
                 -enc_layers 1 \
                 -dec_layers 1 \
                 -rnn_type GRU \
                 -global_attention mlp \
                 -gpuid 0\
                 -batch_size 64 \
                 -epochs 20 \
                 -optim adadelta \
                 -dropout 0.1 \

Loading train dataset from data/med_data_v2.train.1.pt, number of examples: 28800
 * vocabulary size. source = 82; target = 34
Building model...
Intializing model parameters.
NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(82, 300, padding_idx=1)
        )
      )
    )
    (rnn): GRU(300, 250, dropout=0.1, bidirectional=True)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(34, 300, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.1)
    (rnn): StackedGRU(
      (dropout): Dropout(p=0.1)
      (layers): ModuleList(
        (0): GRUCell(800, 500)
      )
    )
    (attn): GlobalAttention(
      (linear_context): Linear(in_features=500, out_features=500)
      (linear_query): Linear(in_features=500, out_features=500)
      (v): Linear(in_features=500

Epoch  3,   150/  450; acc:  96.42; ppl:   1.15; xent:   0.14; 11333 src tok/s; 6963 tgt tok/s;     17 s elapsed
Epoch  3,   200/  450; acc:  96.63; ppl:   1.15; xent:   0.14; 11817 src tok/s; 7037 tgt tok/s;     23 s elapsed
Epoch  3,   250/  450; acc:  96.95; ppl:   1.13; xent:   0.12; 11167 src tok/s; 6908 tgt tok/s;     29 s elapsed
Epoch  3,   300/  450; acc:  97.35; ppl:   1.11; xent:   0.10; 11440 src tok/s; 6837 tgt tok/s;     34 s elapsed
Epoch  3,   350/  450; acc:  97.39; ppl:   1.12; xent:   0.11; 11072 src tok/s; 6850 tgt tok/s;     40 s elapsed
Epoch  3,   400/  450; acc:  96.39; ppl:   1.16; xent:   0.15; 11448 src tok/s; 6877 tgt tok/s;     46 s elapsed
Epoch  3,   450/  450; acc:  96.43; ppl:   1.16; xent:   0.15; 10823 src tok/s; 6552 tgt tok/s;     52 s elapsed
Train perplexity: 1.14065
Train accuracy: 96.7273
Loading valid dataset from data/med_data_v2.valid.1.pt, number of examples: 3200
Validation perplexity: 1.93724
Validation accuracy: 91.3827
Decaying learning 

Epoch  6,   350/  450; acc:  99.67; ppl:   1.01; xent:   0.01; 11323 src tok/s; 7005 tgt tok/s;     40 s elapsed
Epoch  6,   400/  450; acc:  99.55; ppl:   1.02; xent:   0.02; 11747 src tok/s; 7056 tgt tok/s;     45 s elapsed
Epoch  6,   450/  450; acc:  99.70; ppl:   1.01; xent:   0.01; 11018 src tok/s; 6670 tgt tok/s;     51 s elapsed
Train perplexity: 1.01632
Train accuracy: 99.5538
Loading valid dataset from data/med_data_v2.valid.1.pt, number of examples: 3200
Validation perplexity: 2.13263
Validation accuracy: 93.1557
Decaying learning rate to 0.0625

Loading train dataset from data/med_data_v2.train.1.pt, number of examples: 28800
Epoch  7,    50/  450; acc:  99.52; ppl:   1.02; xent:   0.02; 11049 src tok/s; 6776 tgt tok/s;      6 s elapsed
Epoch  7,   100/  450; acc:  99.52; ppl:   1.02; xent:   0.02; 11892 src tok/s; 7141 tgt tok/s;     11 s elapsed
Epoch  7,   150/  450; acc:  99.69; ppl:   1.01; xent:   0.01; 11514 src tok/s; 7075 tgt tok/s;     17 s elapsed
Epoch  7,   200


Loading train dataset from data/med_data_v2.train.1.pt, number of examples: 28800
Epoch 10,    50/  450; acc:  99.68; ppl:   1.01; xent:   0.01; 10907 src tok/s; 6689 tgt tok/s;      6 s elapsed
Epoch 10,   100/  450; acc:  99.69; ppl:   1.01; xent:   0.01; 11747 src tok/s; 7054 tgt tok/s;     11 s elapsed
Epoch 10,   150/  450; acc:  99.79; ppl:   1.01; xent:   0.01; 11406 src tok/s; 7008 tgt tok/s;     17 s elapsed
Epoch 10,   200/  450; acc:  99.73; ppl:   1.01; xent:   0.01; 11749 src tok/s; 6997 tgt tok/s;     23 s elapsed
Epoch 10,   250/  450; acc:  99.79; ppl:   1.01; xent:   0.01; 11182 src tok/s; 6918 tgt tok/s;     28 s elapsed
Epoch 10,   300/  450; acc:  99.81; ppl:   1.01; xent:   0.01; 11619 src tok/s; 6944 tgt tok/s;     34 s elapsed
Epoch 10,   350/  450; acc:  99.84; ppl:   1.01; xent:   0.01; 11313 src tok/s; 6999 tgt tok/s;     40 s elapsed
Epoch 10,   400/  450; acc:  99.74; ppl:   1.01; xent:   0.01; 11528 src tok/s; 6924 tgt tok/s;     45 s elapsed
Epoch 10,   4

Epoch 13,   250/  450; acc:  99.79; ppl:   1.01; xent:   0.01; 11109 src tok/s; 6872 tgt tok/s;     28 s elapsed
Epoch 13,   300/  450; acc:  99.81; ppl:   1.01; xent:   0.01; 11549 src tok/s; 6902 tgt tok/s;     34 s elapsed
Epoch 13,   350/  450; acc:  99.88; ppl:   1.01; xent:   0.01; 11312 src tok/s; 6998 tgt tok/s;     40 s elapsed
Epoch 13,   400/  450; acc:  99.79; ppl:   1.01; xent:   0.01; 11744 src tok/s; 7054 tgt tok/s;     45 s elapsed
Epoch 13,   450/  450; acc:  99.83; ppl:   1.01; xent:   0.01; 11027 src tok/s; 6675 tgt tok/s;     51 s elapsed
Train perplexity: 1.00796
Train accuracy: 99.7819
Loading valid dataset from data/med_data_v2.valid.1.pt, number of examples: 3200
Validation perplexity: 2.29358
Validation accuracy: 93.102
Decaying learning rate to 0.000488281

Loading train dataset from data/med_data_v2.train.1.pt, number of examples: 28800
Epoch 14,    50/  450; acc:  99.68; ppl:   1.01; xent:   0.01; 10824 src tok/s; 6638 tgt tok/s;      6 s elapsed
Epoch 14,  

Loading valid dataset from data/med_data_v2.valid.1.pt, number of examples: 3200
Validation perplexity: 2.29459
Validation accuracy: 93.1071
Decaying learning rate to 6.10352e-05

Loading train dataset from data/med_data_v2.train.1.pt, number of examples: 28800
Epoch 17,    50/  450; acc:  99.68; ppl:   1.01; xent:   0.01; 10982 src tok/s; 6734 tgt tok/s;      6 s elapsed
Epoch 17,   100/  450; acc:  99.71; ppl:   1.01; xent:   0.01; 11821 src tok/s; 7098 tgt tok/s;     11 s elapsed
Epoch 17,   150/  450; acc:  99.81; ppl:   1.01; xent:   0.01; 11349 src tok/s; 6973 tgt tok/s;     17 s elapsed
Epoch 17,   200/  450; acc:  99.75; ppl:   1.01; xent:   0.01; 11675 src tok/s; 6953 tgt tok/s;     23 s elapsed
Epoch 17,   250/  450; acc:  99.81; ppl:   1.01; xent:   0.01; 11116 src tok/s; 6877 tgt tok/s;     29 s elapsed
Epoch 17,   300/  450; acc:  99.83; ppl:   1.01; xent:   0.01; 11518 src tok/s; 6883 tgt tok/s;     34 s elapsed
Epoch 17,   350/  450; acc:  99.85; ppl:   1.01; xent:   0.0

Epoch 20,   150/  450; acc:  99.79; ppl:   1.01; xent:   0.01; 11352 src tok/s; 6975 tgt tok/s;     17 s elapsed
Epoch 20,   200/  450; acc:  99.75; ppl:   1.01; xent:   0.01; 11715 src tok/s; 6976 tgt tok/s;     23 s elapsed
Epoch 20,   250/  450; acc:  99.81; ppl:   1.01; xent:   0.01; 11213 src tok/s; 6937 tgt tok/s;     29 s elapsed
Epoch 20,   300/  450; acc:  99.83; ppl:   1.01; xent:   0.01; 11577 src tok/s; 6919 tgt tok/s;     34 s elapsed
Epoch 20,   350/  450; acc:  99.86; ppl:   1.01; xent:   0.01; 11293 src tok/s; 6986 tgt tok/s;     40 s elapsed
Epoch 20,   400/  450; acc:  99.77; ppl:   1.01; xent:   0.01; 11598 src tok/s; 6967 tgt tok/s;     45 s elapsed
Epoch 20,   450/  450; acc:  99.84; ppl:   1.01; xent:   0.01; 11018 src tok/s; 6669 tgt tok/s;     51 s elapsed
Train perplexity: 1.00797
Train accuracy: 99.783
Loading valid dataset from data/med_data_v2.valid.1.pt, number of examples: 3200
Validation perplexity: 2.29464
Validation accuracy: 93.1071
Decaying learning r

## Train Russian

In [28]:
!python train.py -data data/russian/russian-data \
                 -save_model models/med-model-russ-embed500 \
                 -src_word_vec_size 500 \
                 -tgt_word_vec_size 500 \
                 -encoder_type brnn \
                 -rnn_size 100 \
                 -enc_layers 1 \
                 -dec_layers 1 \
                 -rnn_type GRU \
                 -global_attention mlp \
                 -gpuid 0 \
                 -batch_size 64 \
                 -epochs 20 \
                 -optim adadelta \
                 -dropout 0.1
!cp models/med-model-russ-embed500*e20.pt drive/MED/models

Loading train dataset from data/russian/russian-data.train.1.pt, number of examples: 25600
 * vocabulary size. source = 95; target = 37
Building model...
Intializing model parameters.
NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(95, 500, padding_idx=1)
        )
      )
    )
    (rnn): GRU(500, 50, dropout=0.1, bidirectional=True)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(37, 500, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.1)
    (rnn): StackedGRU(
      (dropout): Dropout(p=0.1)
      (layers): ModuleList(
        (0): GRUCell(600, 100)
      )
    )
    (attn): GlobalAttention(
      (linear_context): Linear(in_features=100, out_features=100)
      (linear_query): Linear(in_features=100, out_features=100)
      (v): Linear(in_feat

Epoch  3,   200/  400; acc:  95.37; ppl:   1.20; xent:   0.18; 13642 src tok/s; 8465 tgt tok/s;     15 s elapsed
Epoch  3,   250/  400; acc:  96.27; ppl:   1.16; xent:   0.15; 13911 src tok/s; 8609 tgt tok/s;     19 s elapsed
Epoch  3,   300/  400; acc:  96.06; ppl:   1.17; xent:   0.15; 13135 src tok/s; 8261 tgt tok/s;     23 s elapsed
Epoch  3,   350/  400; acc:  96.25; ppl:   1.16; xent:   0.15; 13612 src tok/s; 8427 tgt tok/s;     27 s elapsed
Epoch  3,   400/  400; acc:  96.52; ppl:   1.15; xent:   0.14; 13406 src tok/s; 8428 tgt tok/s;     31 s elapsed
Train perplexity: 1.18215
Train accuracy: 95.687
Loading valid dataset from data/russian/russian-data.valid.1.pt, number of examples: 3000
Validation perplexity: 1.14392
Validation accuracy: 96.5249

Loading train dataset from data/russian/russian-data.train.1.pt, number of examples: 25600
Epoch  4,    50/  400; acc:  96.73; ppl:   1.14; xent:   0.13; 13375 src tok/s; 8430 tgt tok/s;      4 s elapsed
Epoch  4,   100/  400; acc:  96

Epoch  7,   100/  400; acc:  98.50; ppl:   1.06; xent:   0.05; 13622 src tok/s; 8404 tgt tok/s;      8 s elapsed
Epoch  7,   150/  400; acc:  98.71; ppl:   1.05; xent:   0.05; 13668 src tok/s; 8539 tgt tok/s;     12 s elapsed
Epoch  7,   200/  400; acc:  98.60; ppl:   1.05; xent:   0.05; 13648 src tok/s; 8469 tgt tok/s;     15 s elapsed
Epoch  7,   250/  400; acc:  98.86; ppl:   1.04; xent:   0.04; 13968 src tok/s; 8644 tgt tok/s;     19 s elapsed
Epoch  7,   300/  400; acc:  98.79; ppl:   1.05; xent:   0.05; 13337 src tok/s; 8388 tgt tok/s;     23 s elapsed
Epoch  7,   350/  400; acc:  98.85; ppl:   1.04; xent:   0.04; 13792 src tok/s; 8538 tgt tok/s;     27 s elapsed
Epoch  7,   400/  400; acc:  98.87; ppl:   1.04; xent:   0.04; 13571 src tok/s; 8532 tgt tok/s;     30 s elapsed
Train perplexity: 1.04745
Train accuracy: 98.7259
Loading valid dataset from data/russian/russian-data.valid.1.pt, number of examples: 3000
Validation perplexity: 1.06916
Validation accuracy: 98.2011
Decaying 

Loading train dataset from data/russian/russian-data.train.1.pt, number of examples: 25600
Epoch 11,    50/  400; acc:  99.03; ppl:   1.03; xent:   0.03; 13364 src tok/s; 8423 tgt tok/s;      4 s elapsed
Epoch 11,   100/  400; acc:  98.90; ppl:   1.04; xent:   0.04; 13770 src tok/s; 8495 tgt tok/s;      8 s elapsed
Epoch 11,   150/  400; acc:  99.03; ppl:   1.03; xent:   0.03; 13836 src tok/s; 8644 tgt tok/s;     11 s elapsed
Epoch 11,   200/  400; acc:  98.97; ppl:   1.04; xent:   0.04; 13719 src tok/s; 8513 tgt tok/s;     15 s elapsed
Epoch 11,   250/  400; acc:  99.19; ppl:   1.03; xent:   0.03; 14134 src tok/s; 8747 tgt tok/s;     19 s elapsed
Epoch 11,   300/  400; acc:  99.07; ppl:   1.03; xent:   0.03; 13421 src tok/s; 8441 tgt tok/s;     23 s elapsed
Epoch 11,   350/  400; acc:  99.21; ppl:   1.03; xent:   0.03; 13847 src tok/s; 8572 tgt tok/s;     27 s elapsed
Epoch 11,   400/  400; acc:  99.17; ppl:   1.03; xent:   0.03; 13421 src tok/s; 8437 tgt tok/s;     30 s elapsed
Train

Loading train dataset from data/russian/russian-data.train.1.pt, number of examples: 25600
Epoch 15,    50/  400; acc:  99.09; ppl:   1.03; xent:   0.03; 13215 src tok/s; 8329 tgt tok/s;      4 s elapsed
Epoch 15,   100/  400; acc:  98.92; ppl:   1.04; xent:   0.04; 13612 src tok/s; 8398 tgt tok/s;      8 s elapsed
Epoch 15,   150/  400; acc:  99.05; ppl:   1.03; xent:   0.03; 13492 src tok/s; 8429 tgt tok/s;     12 s elapsed
Epoch 15,   200/  400; acc:  98.95; ppl:   1.04; xent:   0.04; 13535 src tok/s; 8399 tgt tok/s;     15 s elapsed
Epoch 15,   250/  400; acc:  99.20; ppl:   1.03; xent:   0.03; 13832 src tok/s; 8560 tgt tok/s;     19 s elapsed
Epoch 15,   300/  400; acc:  99.10; ppl:   1.03; xent:   0.03; 13034 src tok/s; 8197 tgt tok/s;     23 s elapsed
Epoch 15,   350/  400; acc:  99.24; ppl:   1.03; xent:   0.03; 13621 src tok/s; 8432 tgt tok/s;     27 s elapsed
Epoch 15,   400/  400; acc:  99.13; ppl:   1.03; xent:   0.03; 13351 src tok/s; 8393 tgt tok/s;     31 s elapsed
Train

Validation perplexity: 1.06314
Validation accuracy: 98.3338
Decaying learning rate to 6.10352e-05

Loading train dataset from data/russian/russian-data.train.1.pt, number of examples: 25600
Epoch 19,    50/  400; acc:  99.06; ppl:   1.03; xent:   0.03; 13292 src tok/s; 8377 tgt tok/s;      4 s elapsed
Epoch 19,   100/  400; acc:  98.97; ppl:   1.04; xent:   0.04; 13560 src tok/s; 8366 tgt tok/s;      8 s elapsed
Epoch 19,   150/  400; acc:  99.06; ppl:   1.03; xent:   0.03; 13603 src tok/s; 8498 tgt tok/s;     12 s elapsed
Epoch 19,   200/  400; acc:  98.98; ppl:   1.04; xent:   0.04; 13561 src tok/s; 8415 tgt tok/s;     15 s elapsed
Epoch 19,   250/  400; acc:  99.18; ppl:   1.03; xent:   0.03; 13963 src tok/s; 8641 tgt tok/s;     19 s elapsed
Epoch 19,   300/  400; acc:  99.19; ppl:   1.03; xent:   0.03; 13491 src tok/s; 8484 tgt tok/s;     23 s elapsed
Epoch 19,   350/  400; acc:  99.19; ppl:   1.03; xent:   0.03; 13902 src tok/s; 8606 tgt tok/s;     27 s elapsed
Epoch 19,   400/  4

In [0]:
#!cp drive/MED/data/russian-data* data/
!ls models

In [30]:
!python translate.py -model models/med-model-russ-embed500_acc_98.33_ppl_1.06_e20.pt \
                    -src data/russian/src-russian-task2-dev \
                    -output preds/pred-russian-embed500 \
                    -replace_unk \
                    -beam_size 12
!cp preds/pred-russian-embed500* drive/MED

Loading model parameters.
average src size 16.2825 1600
PRED AVG SCORE: -0.0179, PRED PPL: 1.0180
